# HTTPサーバー

HTTPサーバーを実行し、GETリクエストを受信したときにカチャカに対して指令を送るサンプルです。
以下のアドレスにカチャカボタンHubなどからGETを送信することで、希望の指令を送ることができます。
```
http://<カチャカのIPアドレス>:26502/run
```

In [ ]:
import asyncio

import kachaka_api
import nest_asyncio
import uvicorn
from fastapi import BackgroundTasks, FastAPI

app = FastAPI()
client = kachaka_api.aio.KachakaApiClient()
lock = asyncio.Lock()


async def send_command():
    await client.speak("カチャカです、よろしくね！")
    print("send_command finished.")


async def run_task():
    if lock.locked():
        print("Another task is already running. Skip.")
        return
    async with lock:
        await send_command()


@app.get("/run")
async def handle_get_run(background_tasks: BackgroundTasks):
    background_tasks.add_task(run_task)
    return {"success": True}


nest_asyncio.apply()  # JupyterLab内でuvicornを走らせるために必要
uvicorn.run(app, host="0.0.0.0", port=26502)